In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('/opt/ml/detection/models/Swin/swinS_cascade.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/swinS_cascade'

cfg.model.roi_head.bbox_head[0].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

starts k-fold with /opt/ml/detection/dataset/SM_train1.json


In [4]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [5]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [6]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-03-23 09:38:49,863 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_small_patch4_window7_224.pth
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_small_patch4_window7_224.pth" to /opt/ml/.cache/torch/hub/checkpoints/swin_small_patch4_window7_224.pth


  0%|          | 0.00/191M [00:00<?, ?B/s]

2022-03-23 09:39:04,859 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-03-23 09:39:04,935 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-03-23 09:39:04,951 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-03-23 09:39:05,160 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-03-23 09:39:05,383 - mmdet - INFO - initialize Sh

In [2]:
import wandb

wandb.init(project="swinS-cascade", entity="cv-09")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: updaun (use `wandb login --relogin` to force relogin)


In [7]:
# 모델 학습

train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2022-03-23 09:39:32,803 - mmdet - INFO - Start running, host: root@81692cdfc99c, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/swinS_cascade
2022-03-23 09:39:32,804 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL 

2022-03-23 09:39:53,168 - mmdet - INFO - Epoch [1][10/1221]	lr: 6.350e-06, eta: 11:35:36, time: 0.855, data_time: 0.230, memory: 6322, loss_rpn_cls: 0.6689, loss_rpn_bbox: 0.0177, s0.loss_cls: 1.8565, s0.acc: 55.0391, s0.loss_bbox: 0.0097, s1.loss_cls: 0.5733, s1.acc: 91.6016, s1.loss_bbox: 0.0021, s2.loss_cls: 0.4066, s2.acc: 73.9062, s2.loss_bbox: 0.0003, loss: 3.5352, grad_norm: 40.0518
2022-03-23 09:39:58,801 - mmdet - INFO - Epoch [1][20/1221]	lr: 7.850e-06, eta: 9:36:56, time: 0.563, data_time: 0.007, memory: 6322, loss_rpn_cls: 0.6367, loss_rpn_bbox: 0.0239, s0.loss_cls: 0.4732, s0.acc: 96.4795, s0.loss_bbox: 0.0457, s1.loss_cls: 0.1431, s1.acc: 97.8613, s1.loss_bbox: 0.0119, s2.loss_cls: 0.1027, s2.acc: 98.6719, s2.loss_bbox: 0.0013, loss: 1.4385, grad_norm: 14.2510
2022-03-23 09:40:04,863 - mmdet - INFO - Epoch [1][30/1221]	lr: 9.350e-06, eta: 9:08:55, time: 0.606, data_time: 0.007, memory: 6322, loss_rpn_cls: 0.5886, loss_rpn_bbox: 0.0229, s0.loss_cls: 0.3719, s0.acc: 94.2480

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
learning_rate,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▅▆█▆▇▅▇▆▆▇▅▆█▄▄▃▄▃▄▃▃▃▃▂▅▃▇▄▂▃▂▄▁▂▂▄▂▁▂▂
train/loss,█▇▇▅▆▄▆▅▄▅▄▃▄▃▃▂▂▃▄▂▂▃▁▂▂▂▂▂▁▂▂▂▂▁▁▂▁▁▂▂
train/loss_rpn_bbox,█▆▆▄▆▃▇▅▄▆▃▃▂▆▃▂▃▄▅▂▂▃▁▃▃▄▃▃▃▃▄▂▄▃▂▂▂▃▃▄
train/loss_rpn_cls,█▇▆▄▅▄▅▄▃▄▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▂▂
train/s0.acc,▁▁▂▃▂▅▂▄▅▄▅▅▅▅▆▇▆▆▅▇▆▆█▇▆▇▆▇▇▇▆▇▇▇█▇██▇▇
train/s0.loss_bbox,█▇▇▆▇▄█▅▄▅▄▃▄▃▃▂▃▄▄▃▃▃▁▂▃▃▃▂▂▃▃▂▂▁▁▂▁▁▂▂
train/s0.loss_cls,█▇▆▅▆▄▅▄▄▅▄▃▃▃▃▂▂▃▄▂▂▂▁▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▂
train/s1.acc,▁▁▂▄▂▆▂▄▆▄▅▅▅▅▆▇▇▆▆▇▇▇██▇▇▇▇▇▇▇▇███▇██▇▇
